In [1]:
from bokeh.plotting import figure, save
from bokeh.models import (ColumnDataSource, HoverTool, LogColorMapper,GMapPlot,
                          GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool,
                          WheelZoomTool, BoxSelectTool)
import numpy as np
from bokeh.plotting import show, output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import json, requests
from pandas.io.json import json_normalize
import pandas as pd

In [3]:
def query_site(url, params):
    # Queries the Water Quality Sample Data of data.austintexas.gov.
    # A json document is returned by the query.
    r = requests.get(url, params)
    print("Requesting", r.url)
    
    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()

In [4]:
base_url = 'https://data.austintexas.gov/resource/v7et-4fvp.json?$limit=50000'
params = {'parameter': 'E COLI BACTERIA', 'unit': 'MPN/100ML'}
data = query_site(base_url, params)
df = json_normalize(data)

('Requesting', u'https://data.austintexas.gov/resource/v7et-4fvp.json?$limit=50000&parameter=E+COLI+BACTERIA&unit=MPN%2F100ML')


In [5]:
ecoli_df = ecoli_df = df[['sample_date','site_name','parameter','result','unit','lat_dd_wgs84','lon_dd_wgs84']]
ecoli_df.rename(columns={'lat_dd_wgs84': 'latitude', 'lon_dd_wgs84': 'longitude'}, inplace=True)
ecoli_df['sample_date'] = pd.to_datetime(ecoli_df['sample_date']) 
mask = (ecoli_df['sample_date'] > '2016-01-01') 
ecoli_df = ecoli_df.loc[mask]
ecoli_df['result'] = ecoli_df['result'].astype('float64') 
ecoli_df['latitude'] = ecoli_df['latitude'].astype('float64')
ecoli_df['longitude'] = ecoli_df['longitude'].astype('float64')
ecoli_score = ecoli_df.groupby(['site_name']).mean()
ecoli_score.reset_index(inplace=True)
ecoli_score.head()

/Users/Ridhi/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
/Users/Ridhi/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,site_name,result,latitude,longitude
0,Backdoor Spring,5.345000,30.259493,-97.823701
1,Barton Creek @ Hwy 71 Downstream of Little Barton,11.963333,30.296229,-97.926093
2,Barton Creek @ Ogletree Pool,19.510000,30.302273,-97.868544
3,Barton Creek @ Shield Ranch Pool,96.400000,30.269785,-97.973509
4,Barton Creek @ Stark Pool,26.350000,30.244922,-98.125694


In [6]:
ecoli_score['x'] = ecoli_score['longitude']
ecoli_score['y'] = ecoli_score['latitude']
k = 6378137.0
ecoli_score['x'] = ecoli_score['x'].apply(lambda x: (x*k * np.pi/180.0))
ecoli_score['y'] = ecoli_score['y'].apply(lambda y: np.log(np.tan((90.0 + y) * (np.pi)/360.0))*k)
ecoli_score['Unit']='MPN/100ML'     
ecoli_score.head()

,site_name,result,latitude,longitude,x,y,Unit
0,Backdoor Spring,5.345000,30.259493,-97.823701,-1.088968e+07,3.536949e+06,MPN/100ML
1,Barton Creek @ Hwy 71 Downstream of Little Barton,11.963333,30.296229,-97.926093,-1.090108e+07,3.541684e+06,MPN/100ML
2,Barton Creek @ Ogletree Pool,19.510000,30.302273,-97.868544,-1.089468e+07,3.542464e+06,MPN/100ML
3,Barton Creek @ Shield Ranch Pool,96.400000,30.269785,-97.973509,-1.090636e+07,3.538276e+06,MPN/100ML
4,Barton Creek @ Stark Pool,26.350000,30.244922,-98.125694,-1.092330e+07,3.535071e+06,MPN/100ML


In [7]:
ecoli_score_unsafe = ecoli_score.loc[ecoli_score['result'] > 400.0]
ecoli_score_unsafe['Warning'] = 'Unsafe to Swim'
ecoli_score_safe = ecoli_score.loc[ecoli_score['result'] <= 400.0]
ecoli_score_safe['Warning'] = 'Safe to Swim'
ecoli_score_unsafe.head()

/Users/Ridhi/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Ridhi/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,site_name,result,latitude,longitude,x,y,Unit,Warning
18,Blunn Creek @ Long Bow (Preserve at Little Bri...,500.300000,30.233593,-97.748006,-1.088126e+07,3.533612e+06,MPN/100ML,Unsafe to Swim
20,Blunn Creek Upstream of Big Stacy Pool,1027.316667,30.240127,-97.747561,-1.088121e+07,3.534453e+06,MPN/100ML,Unsafe to Swim
23,Bull Creek @ Bull Creek Dist Park (Lakewood Dr),473.746429,30.366375,-97.787065,-1.088561e+07,3.550731e+06,MPN/100ML,Unsafe to Swim
24,Bull Creek @ Loop 360 First Crossing,462.302941,30.371671,-97.784930,-1.088537e+07,3.551415e+06,MPN/100ML,Unsafe to Swim
27,Buttermilk Creek @ Providence Ave,1710.966667,30.331736,-97.694379,-1.087529e+07,3.546263e+06,MPN/100ML,Unsafe to Swim


In [8]:
from bokeh.plotting import figure
from bokeh.tile_providers import WMTSTileSource
# Austin = ((-10898752, -10855820), (3525750, 3550837))
USA = x_range,y_range = ((-10898752, -10855820), (3525750, 3550837))
TOOLS="pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

fig = figure(title = "Water Quality Map (Here Red-Unsafe and Green-Safe)",tools=TOOLS, x_range=x_range, y_range=y_range,plot_width=1000, plot_height=800)
fig.axis.visible = False
url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
attribution = "Map tiles by WikiMedia"
fig.add_tile(WMTSTileSource(url=url, attribution=attribution))
dfsource1 = ColumnDataSource(data=ecoli_score_safe)
fig.circle('x', 'y',source=dfsource1,color="green" )
dfsource2 = ColumnDataSource(data=ecoli_score_unsafe)
fig.circle('x','y',source=dfsource2,color="red")
hover = HoverTool(tooltips=[
    ("Site Name", "@site_name"),
    ("Ecoli_Level", "@result"),
    ("Unit", "@Unit"),
    ("Warning","@Warning")
])
fig.add_tools(hover)
show(fig)